# 🍫 Cocoa Disease Detection with YOLOv8
## Training on Google Colab

This notebook trains a YOLOv8 model to detect cocoa diseases:
- **Healthy** cocoa pods
- **Phytophthora** disease
- **Monilia** disease

**Dataset:** 312 images (1,591 labeled objects)

---

## 📋 Before You Start - Checklist

Make sure you've completed these steps:
- ✅ Downloaded dataset from Dataset Ninja
- ✅ Converted to YOLO format using `convert_supervisely_to_yolo.py`
- ✅ Uploaded to Google Drive at: `MyDrive/cocoa-disease-detection/dataset/`
- ✅ Dataset structure has: `train/`, `val/`, `test/` folders and `data.yaml`

## Step 1: Check GPU Availability

**⚠️ IMPORTANT:** Before running this cell, enable GPU:
1. Click **Runtime** → **Change runtime type**
2. Select **Hardware accelerator**: **T4 GPU**
3. Click **Save**
4. Then run this cell to verify

In [ ]:
# Check GPU availability
!nvidia-smi

print("\n" + "="*60)
print("GPU Check Complete!")
print("If you see GPU info above, you're good to go! ✅")
print("If you see an error, go to Runtime → Change runtime type → GPU")
print("="*60)

## Step 2: Install Required Packages

In [ ]:
# Install YOLOv8 (Ultralytics)
!pip install ultralytics -q

# Import libraries
import os
import shutil
from pathlib import Path
import yaml
import torch
from ultralytics import YOLO
from IPython.display import Image, display
import matplotlib.pyplot as plt

print("\n✅ Packages installed successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Step 3: Mount Google Drive

This will connect to your Google Drive where you uploaded the dataset.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("\n✅ Google Drive mounted successfully!")
print("📁 Your Drive is accessible at: /content/drive/MyDrive/")

## Step 4: Verify Dataset Location

Let's make sure your dataset is in the right place!

In [ ]:
# Dataset paths
DATASET_DIR = '/content/drive/MyDrive/cocoa-disease-detection/dataset'
PROJECT_DIR = '/content/drive/MyDrive/cocoa-disease-detection'

print("🔍 Checking dataset location...\n")

# Check if dataset directory exists
if os.path.exists(DATASET_DIR):
    print(f"✅ Dataset directory found: {DATASET_DIR}")
    
    # Check data.yaml
    yaml_path = f"{DATASET_DIR}/data.yaml"
    if os.path.exists(yaml_path):
        print("✅ data.yaml found")
    else:
        print("❌ data.yaml NOT found - this is required!")
    
    # Check train/val/test folders
    for split in ['train', 'val', 'test']:
        img_dir = f"{DATASET_DIR}/{split}/images"
        label_dir = f"{DATASET_DIR}/{split}/labels"
        
        if os.path.exists(img_dir) and os.path.exists(label_dir):
            num_images = len([f for f in os.listdir(img_dir) if f.endswith('.jpg')])
            num_labels = len([f for f in os.listdir(label_dir) if f.endswith('.txt')])
            print(f"✅ {split:5} - Images: {num_images:3}, Labels: {num_labels:3}")
        else:
            print(f"❌ {split} folder missing images or labels")
    
    print("\n📊 Expected counts: Train=249, Val=31, Test=32")
    print("\n✨ If all checks passed, you're ready to train!")
else:
    print(f"❌ Dataset directory NOT found: {DATASET_DIR}")
    print("\n🔧 Please check:")
    print("   1. Did you upload to the correct folder in Google Drive?")
    print("   2. Path should be: MyDrive/cocoa-disease-detection/dataset/")
    print("   3. Dataset should contain: train/, val/, test/, data.yaml")

## Step 5: Update data.yaml with Correct Paths

In [ ]:
# Update data.yaml with correct paths
yaml_content = f"""path: {DATASET_DIR}
train: train/images
val: val/images
test: test/images

names:
  0: healthy
  1: phytophthora
  2: monilia

nc: 3
"""

yaml_path = f"{DATASET_DIR}/data.yaml"
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print("✅ data.yaml updated with Colab paths")

## Step 6: Initialize YOLOv8 Model

Choose nano (n) for speed or small (s) for better accuracy.

In [ ]:
# Choose model size
MODEL_SIZE = 'yolov8n.pt'  # Options: yolov8n.pt, yolov8s.pt, yolov8m.pt

print(f"🚀 Initializing {MODEL_SIZE}...")
model = YOLO(MODEL_SIZE)
print(f"\n✅ Model loaded: {MODEL_SIZE}")

## Step 7: Configure Training Parameters

In [ ]:
# Training configuration
EPOCHS = 100
IMG_SIZE = 640
BATCH_SIZE = 16
PATIENCE = 20

print("⚙️ Training Configuration:")
print(f"   Epochs: {EPOCHS}")
print(f"   Image Size: {IMG_SIZE}")
print(f"   Batch Size: {BATCH_SIZE}")
print(f"\n💡 Estimated training time: 30-60 minutes")

## Step 8: Train the Model! 🚀

This will take 30-60 minutes. Keep the browser tab active!

In [ ]:
import time

print("🚀 Starting training...\n")
print("⏰ Started at:", time.strftime("%Y-%m-%d %H:%M:%S"))

# Train the model
results = model.train(
    data=yaml_path,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    patience=PATIENCE,
    save=True,
    device=0,
    project=PROJECT_DIR,
    name='yolov8_training',
    exist_ok=True,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10.0,
    translate=0.1,
    scale=0.5,
    flipud=0.5,
    fliplr=0.5,
    mosaic=1.0,
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.01,
    verbose=True,
    plots=True,
    save_period=10
)

print("\n✅ Training completed!")
print("⏰ Finished at:", time.strftime("%Y-%m-%d %H:%M:%S"))

## Step 9: View Training Results

In [ ]:
results_dir = f"{PROJECT_DIR}/yolov8_training"

# Display training curves
if os.path.exists(f"{results_dir}/results.png"):
    print("📈 Training Curves:")
    display(Image(filename=f"{results_dir}/results.png", width=800))

# Display confusion matrix
if os.path.exists(f"{results_dir}/confusion_matrix.png"):
    print("\n🔍 Confusion Matrix:")
    display(Image(filename=f"{results_dir}/confusion_matrix.png", width=600))

## Step 10: Evaluate on Test Set

In [ ]:
# Load best model
best_model = YOLO(f"{results_dir}/weights/best.pt")

print("🧪 Evaluating on test set...\n")

# Validate
metrics = best_model.val(data=yaml_path, split='test', imgsz=IMG_SIZE, batch=BATCH_SIZE)

print("\n📊 Test Set Performance")
print("="*60)
print(f"mAP50:     {metrics.box.map50:.4f} (Target: >0.70)")
print(f"mAP50-95:  {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall:    {metrics.box.mr:.4f}")

class_names = ['healthy', 'phytophthora', 'monilia']
print("\n📋 Per-Class Performance:")
for i, name in enumerate(class_names):
    print(f"{name}: P={metrics.box.p[i]:.3f} R={metrics.box.r[i]:.3f} mAP50={metrics.box.ap50[i]:.3f}")

## Step 11: Export Model

In [ ]:
print("📦 Exporting model...\n")

# Export to ONNX
onnx_path = best_model.export(format='onnx', dynamic=True)
print(f"✅ ONNX: {onnx_path}")

# Export to TorchScript
ts_path = best_model.export(format='torchscript')
print(f"✅ TorchScript: {ts_path}")

print("\n✅ Exports complete! Files saved in Google Drive.")

## 🎉 Training Complete!

### Your files are in Google Drive:
`MyDrive/cocoa-disease-detection/yolov8_training/`

**Important files:**
- `weights/best.pt` - Best model (use this!)
- `best.onnx` - ONNX export
- `results.png` - Training curves
- `confusion_matrix.png` - Performance matrix

### Using your model:
```python
from ultralytics import YOLO
model = YOLO('best.pt')
results = model.predict('image.jpg')
```

**Next:** Download `best.pt` from Drive and integrate into your app! 🚀